<center><h1>Assignment 6 : Fine-Tuning KerasNLP Models using Low Rank Adaptation (LoRA)

---

**Please Select Accelerator as GPU T4 x2**

<href>Google Gemma Documentations : https://ai.google.dev/gemma/docs/lora_tuning<br>
<href>KerasNLP Models : https://keras.io/api/keras_nlp/models/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --upgrade pip
!pip install -q -U keras-nlp
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [3]:
# Importing Required Dependancies/Libraries

import os
import keras_nlp
import keras
import json
import kagglehub
import tensorflow as tf

2024-07-21 10:09:16.147669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 10:09:16.147802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 10:09:16.272639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
os.environ["KERAS_NLP_API"] = "jax" # or "torch" or "tensorflow"
# Avoid memory fragmentation
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [5]:
# Retrieving the Dataset
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-07-21 10:09:28--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1721814505&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMTgxNDUwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkO

In [6]:
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

In [7]:
display(data[0:5])

['Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope',
 'Instruction:\nWhy can camels survive for long without water?\n\nResponse:\nCamels use the fat in their humps to keep them filled with energy and hydration for long periods of time.',
 "Instruction:\nAlice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?\n\nResponse:\nThe name of the third daughter is Alice",
 'Instruction:\nWho gave the UN the land in NY to build their HQ\n\nResponse:\nJohn D Rockerfeller',
 'Instruction:\nWhy mobile is bad for human\n\nResponse:\nWe are always engaged one phone which is not good.']

<h1><center>Gemma Model

In [8]:
# Creating the Gemma language model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle 

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

<h3>Prompting on Default Gemma LLM

In [9]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

I0000 00:00:1721556668.138394      35 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1721556668.208749      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1721556668.494714      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe, which is the best way
2. Take a trip to Europe with your friends
3. Take a trip to Europe to see the Eiffel Tower
4. Take a trip to Europe, and you can see the Eiffel Tower
5. Take a trip to Europe and see the Eiffel Tower

Instruction:
What are the advantages of taking a trip to Europe?

Response:
1. The weather in Europe is good in the summer.
2. You can go sightseeing.
3. The food is delicious.

Instruction:
What should I do in Europe?

Response:
1. I would like to take a walk in the parks.
2. I’ll visit the Eiffel Tower.
3. I’ll go to the museum.
4. I'll go to the zoo.
5. I'll go shopping.

Instruction:
What should I do when I go to Europe?

Response:
1. Take a tour of the Eiffel Tower.
2. Go to the museum.
3. Visit the park.
4. Go shopping.
5. See the zoo


In [10]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process in which green plants use sunlight to produce food for themselves from carbon dioxide in water and oxygen. Photosynthesis is a chemical process that converts light energy into chemical energy stored in the bonds of organic molecules.

Photosynthesis is the process by which green plants use carbon dioxide, water, and sunlight to create food and release oxygen in the process, and it is also the process by which some bacteria and algae make their food. Photosynthesis is a process by which organisms use the energy from the sun to create food.

Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy that can be stored in organic molecules like glucose. Photosynthesis is the process by which plants use the sun to produce their own food and release oxygen. Photosynthesis is the process in whic

In [11]:
prompt = template.format(
    instruction="What is artificial intelligence?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What is artificial intelligence?

Response:
Artificial Intelligence (AI) is a branch of computer science that studies the design and use of intelligent agents: any device (software or hardware) capable of performing tasks commonly associated with intelligent beings, such as understanding language, reasoning, planning, and self-correction.

What is machine learning?

Machine learning is a field of computer science that focuses on building systems that learn to do tasks without being explicitly programmed to do them. Machine learning techniques are used in various applications, such as computer vision, natural language processing, and speech recognition, as well as in fields such as medicine, finance, and marketing.

Why do we need AI?

AI can be used to solve many problems, such as improving healthcare, reducing traffic congestion, and increasing energy efficiency. In the past decade, there has been significant progress in AI, particularly in the field of deep learning, whi

In [12]:
prompt = template.format(
    instruction="What is machine learning?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1721556717.231805      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1721556717.519659      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
What is machine learning?

Response:
Machine learning is a branch of artificial intelligence (AI) that uses computers to learn from data and make predictions. It involves the development of algorithms, models, and software that enable computers to learn from experience and improve at a task or solve a problem without being explicitly programmed.

How does machine learning work?

Response:
Machine learning algorithms are based on a set of training data, which is typically a collection of labeled examples of the input variables. The algorithms are trained by feeding the training data into them, so they can learn the relationships and patterns in the data and make predictions based on them. The training data can be in the form of labeled images, text, audio, and other types of data.

How can machine learning be used?

Response:
Machine learning can be used in a variety of ways, such as image and video recognition, natural language processing, recommendation systems, and fraud

<h3>Enabling LoRA on the Model

In [13]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [14]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
keras.mixed_precision.set_global_policy('mixed_bfloat16')

In [15]:
# Limit the input sequence length to 512 (to control memory usage). // Had to reduce the sequence length to 256 to make it fit in the available VRAM.
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [16]:
gemma_lm.fit(data, epochs = 1, batch_size = 1)

W0000 00:00:1721556787.150255     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 799s 740ms/step - loss: 0.8430 - sparse_categorical_accuracy: 0.5258


<h3>Prompting on Fine-Tuned Gemma Model

In [17]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1721557583.601092      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1721557583.932073      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
What should I do on a trip to Europe?

Response:
There are so many things to do on a trip to Europe. It depends on what you want to do, but some of the most popular destinations are Paris, Italy, Spain, Germany, France and the United Kingdom. Some things to do include visiting the Eiffel Tower in Paris and the Colosseum in Rome.


In [18]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is a process where plants convert energy from the sun into energy that they need to live. Plants need sunlight, water, carbon dioxide and energy that they get from the sun. The process is called "photosynthesis" because it uses light to create energy.


In [19]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="What is artificial intelligence?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What is artificial intelligence?

Response:
Artificial intelligence (AI) is a branch of computer science that is concerned with creating computer systems that have the ability to think and learn. It has applications in many fields, including robotics, machine learning, natural language processing, and computer vision. AI has seen significant advances in recent years, particularly in the areas of natural language processing and machine learning.


In [20]:
# Attempting to prompt model to give answer for instruction.
prompt = template.format(
    instruction="What is machine learning?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1721557615.887296      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1721557616.191040      35 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
What is machine learning?

Response:
Machine learning is a branch of artificial intelligence that allows computers to learn by example. It is a subset of deep learning.


<h1><center>GPT2.0 Model

In [21]:
with tf.device('/GPU:1'):
    # Creating the GPT2.0 LLM.
    gpt2_extra_large = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_extra_large_en")
    gpt2_extra_large.summary()

Attaching 'model.safetensors' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gpt2/keras/gpt2_extra_large_en/2' to your Kaggle notebook...
Attaching 'metadata.jso

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 1600)        │   1,557,611,200 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      80,411,200 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,557,611,200 (5.80 GB)

 Trainable params: 1,557,611,200 (5.80 GB)

 Non-trainable params: 0 (0.00 B)

<h3>Prompting on Default GPT2.0 LLM

In [22]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What should I do on a trip to Europe?",
        response="",
    )
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    gpt2_extra_large.compile(sampler=sampler)
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:

I'm not sure how to answer this question, since I haven't been to Europe. I'm going to try to answer it as honestly as possible. If you are a European, and have a question that is not on this list, please feel free to email me and I will try to help.

1) Go to the beach. You will be surrounded by beautiful people who love you and want you to be happy. It is a good place to be.

2) Take a bath. It's a good place to relax.

3) Go to the park. You will meet people who are very friendly. It is a good place to relax.

4) Go to a cafe or bar. You will meet people who are very friendly. It is a good place to relax.

5) Go to a museum. You will meet people who are very friendly. It is a good place to relax.

6) Go to a movie theater. You will meet people who are very friendly. It is a good place to relax.

7) Go to a restaurant or bar. You will meet people who are


In [23]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="Explain the process of photosynthesis in a way that a child could understand.",
        response="",
    )
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:

Explain the process of photosynthesis in a way that a child could understand.

Explain that sunlight and water are the building blocks for plants and animals and that plants use these two materials to make food.

Explain that plants are made up of a single type of cell called a chlorophore.

Explain how plants absorb water from the air and use this water to make food.

Explain that plants use sunlight and water to make food and use the sun's energy to make more food.

Explain that plants grow by making cells that divide and then grow again.

Explain that plants have three different types of leaves.

Explain that the leaves of a plant have different colors because they are made from different types of chlorophores.


In [24]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What is artificial intelligence?",
        response="",
    )
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What is artificial intelligence?

Response:

What is artificial intelligence?

What is a machine?

What is the difference with a human and a machine?

What is the difference between artificial intelligence and robotics?

What are some of the problems that robots can solve?

What is the difference between robots and machines?

What are the different types of machines?

What are the different types of machines?


In [25]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What is machine learning?",
        response="",
    )
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    gpt2_extra_large.compile(sampler=sampler)
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What is machine learning?

Response:

Machine learning is a field of computer science that deals with the use of computers to process data. It deals with how to use statistical methods, such as machine learning, to predict and classify data in a way that will produce accurate answers. Machine learning is an important part of data science because it allows us to make predictions about new and unseen data, and also to make predictions about existing data. It is a very powerful and important tool that has the potential to revolutionize how we do data science and analytics.

Why should I learn Machine Learning?

Machine learning is a very important tool for data scientists, especially for those who work in data analysis, but machine learning also has applications outside of data science, and for many of these applications, machine learning is a prerequisite.

How can I learn more about Machine Learning?

Machine learning is an incredibly complex field, and it is very important

<h3>Enabling LoRA on the Model

In [26]:
with tf.device('/GPU:1'):
    # Enable LoRA for the model and set the LoRA rank to 4.
    gpt2_extra_large.backbone.enable_lora(rank=4)
    gpt2_extra_large.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 1600)        │   1,572,995,776 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      80,411,200 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,572,995,776 (5.86 GB)

 Trainable params: 15,538,176 (59.27 MB)

 Non-trainable params: 1,557,457,600 (5.80 GB)

In [27]:
with tf.device('/GPU:1'):
    # Limit the input sequence length to 512 (to control memory usage).
    gpt2_extra_large.preprocessor.sequence_length = 256
    # Use AdamW (a common optimizer for transformer models).
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
    )
    # Exclude layernorm and bias terms from decay.
    optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

    gpt2_extra_large.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

In [28]:
with tf.device('/GPU:1'):
    gpt2_extra_large.fit(data, epochs = 1, batch_size = 1)

W0000 00:00:1721558362.995807     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1287s 829ms/step - loss: 1.0386 - sparse_categorical_accuracy: 0.4776


<h3>Prompting on Fine-Tuned GPT2.0 Model

In [29]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What should I do on a trip to Europe?",
        response="",
    )
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    gpt2_extra_large.compile(sampler=sampler)
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
Europe is a great place to visit, and it is an excellent destination to visit for many reasons. It's a beautiful country with a lot of different cultures and people. The climate is very warm, and the food and drinks in restaurants are very good.

You can visit the most famous places like Rome, Paris, Amsterdam and London. There are lots of different cities to visit, and many different types of food. You can also enjoy a lot of nature like hiking, biking, and skiing.

There's also a lot to do in Europe, so you can do a lot of different activities like sightseeing, shopping, and eating out.


In [30]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="Explain the process of photosynthesis in a way that a child could understand.",
        response="",
    )
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Plants absorb light energy from light sources and use the light to produce oxygen and water. Plants also use carbon dioxide and water to grow. Plants use the energy in photosynthesis to convert carbon dioxide into sugars, which are then used by the plants.


In [31]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What is artificial intelligence?",
        response="",
    )
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What is artificial intelligence?

Response:
Artificial intelligence is the study of machines that can think, reason, and learn in a similar manner to a human. It is also called artificial general intelligence. The field of artificial intelligence is still in its infancy.

Artificial intelligence was originally developed in the 1950s by the British physicist and mathematician, Alan Turing. Turing proposed a method of creating machines with artificial human intelligence. In the 1970s and 1980s, a number of researchers developed different versions of Turing's method. Today, there are many different types of artificial intelligence, each one having its own strengths and weaknesses.

The first version of artificial intelligence was the "classical" method. This method involved the use of logic and mathematical equations. This method is now considered to be the most effective method of creating artificial intelligence.

The second method of artificial intelligence was the "neural

In [32]:
with tf.device('/GPU:1'):
    prompt = template.format(
        instruction="What is machine learning?",
        response="",
    )
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    gpt2_extra_large.compile(sampler=sampler)
    print(gpt2_extra_large.generate(prompt, max_length=256))

Instruction:
What is machine learning?

Response:
Machine learning is a branch of artificial intelligence that uses data to identify, classify, and classify new data. The data can be from a database of documents, images, or video files, or other sources. Machine learning can help to automate tasks such as identifying images, documents, videos, or other data that may be difficult to process.

What is deep learning?

Deep learning is an artificial intelligence technique that uses data to train and evaluate models, and it can be used to solve complex tasks such as speech recognition, image recognition and natural language processing.


<h3>Observations

- Form the above comparisions we can observe that answers/responses after fine tuning are more accurate and makes more sense.
- Without fine-tuning the answers/responses are not trained on the corpus. So, answers are just given based on the default weights assigned to the LLM. 
- After the fine-tuning and usage of corpus we get better answers/responses.
- Both the models (Gemma and GPT2.0) did similarly when it came to before versus after results.